<a href="https://colab.research.google.com/github/ann04ka/CV_exercises/blob/main/ex2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 2


## Рассмотрим архитектуры первых сверточных нейронных сетей: LeNet, Alexnet, VGG

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [ ]:
class LeNet(nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, input):
        c1 = F.relu(self.conv1(input))
        s2 = F.max_pool2d(c1, (2, 2))
        c3 = F.relu(self.conv2(s2))
        s4 = F.max_pool2d(c3, 2)
        s4 = torch.flatten(s4, 1)
        f5 = F.relu(self.fc1(s4))
        f6 = F.relu(self.fc2(f5))
        output = self.fc3(f6)
        return output

In [ ]:
class AlexNet(nn.Module):

    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(256 * 4 * 4, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)
        self.dropout = nn.Dropout()

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Определим функции для обучения и тестирования моделей

In [ ]:
def train_model(model, trainloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99 and (epoch + 1) % 5 == 0:
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0
    print('Finished Training')

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def evaluate_model(model, testloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f'Accuracy of the network on the test images: {round(accuracy, 3) * 100}%')
    print(f'F1 Score of the network on the test images: {round(f1, 3)}')


## Подготовим обучающую и тестовую выборку, возьмем CIFAR10

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Обучим все модели с одинаковми гиперпараметрами

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11')
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

train_model(model, trainloader, criterion, optimizer)

evaluate_model(model, testloader, device)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


[Epoch 5, Batch 100] loss: 0.816
[Epoch 5, Batch 200] loss: 0.828
[Epoch 5, Batch 300] loss: 0.828
[Epoch 5, Batch 400] loss: 0.782
[Epoch 5, Batch 500] loss: 0.757
[Epoch 10, Batch 100] loss: 0.233
[Epoch 10, Batch 200] loss: 0.256
[Epoch 10, Batch 300] loss: 0.309
[Epoch 10, Batch 400] loss: 0.291
[Epoch 10, Batch 500] loss: 0.324
Finished Training
Accuracy of the network on the test images: 75.8%
F1 Score of the network on the test images: 0.757


In [ ]:
model = LeNet()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

train_model(model, trainloader, criterion, optimizer)

evaluate_model(model, testloader, device)

[Epoch 5, Batch 100] loss: 1.145
[Epoch 5, Batch 200] loss: 1.144
[Epoch 5, Batch 300] loss: 1.138
[Epoch 5, Batch 400] loss: 1.130
[Epoch 5, Batch 500] loss: 1.131
[Epoch 10, Batch 100] loss: 0.839
[Epoch 10, Batch 200] loss: 0.875
[Epoch 10, Batch 300] loss: 0.862
[Epoch 10, Batch 400] loss: 0.887
[Epoch 10, Batch 500] loss: 0.880
Finished Training
Accuracy of the network on the test images: 63.7%
F1 Score of the network on the test images: 0.637


In [ ]:
model = AlexNet()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

train_model(model, trainloader, criterion, optimizer)

evaluate_model(model, testloader, device)

[Epoch 5, Batch 100] loss: 1.047
[Epoch 5, Batch 200] loss: 1.020
[Epoch 5, Batch 300] loss: 1.015
[Epoch 5, Batch 400] loss: 0.980
[Epoch 5, Batch 500] loss: 0.945
[Epoch 10, Batch 100] loss: 0.532
[Epoch 10, Batch 200] loss: 0.534
[Epoch 10, Batch 300] loss: 0.526
[Epoch 10, Batch 400] loss: 0.534
[Epoch 10, Batch 500] loss: 0.548
Finished Training
Accuracy of the network on the test images: 77.7%
F1 Score of the network on the test images: 0.772
